<a href="https://colab.research.google.com/github/Intruz01/DW_Matrix_roadSignRecognition/blob/master/matrix3_day4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
cd "/content/drive/My Drive/Colab Notebooks/Matrix/Matrix_three_road_signs_recognition/DW_Matrix_roadSignRecognition"

/content/drive/My Drive/Colab Notebooks/Matrix/Matrix_three_road_signs_recognition/DW_Matrix_roadSignRecognition


In [0]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
#to have everytimme the same score
if y_train.ndim == 1: y_train = to_categorical(y_train)

if y_test.ndim == 1: y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
model = Sequential ([
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=input_shape),

    Flatten(),

    Dense(num_classes, activation='softmax')
])

#model.summary()

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.fit(X_train, y_train)

1088/1088 [==============================] - 4s 3ms/step - loss: 5.5256 - accuracy: 0.7899


In [0]:
def get_cnn_v1(num_classes, input_shape):
  return Sequential ([
      Conv2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=input_shape),

      Flatten(),

     Dense(num_classes, activation='softmax'),
  ])

def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  
  logdir = os.path.join('logs', datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train, 
      y_train,
      batch_size = params_fit.get('bach_size', 128),
      epochs = params_fit.get('epochs', 5),
      verbose = params_fit.get('verbose', 1),
      validation_data = params_fit.get('validation_data', (X_train, y_train)),
      callbacks = [tensorboard_callback]
  )
  
  return model

## Day 4 - improvings

In [0]:
model = get_cnn_v1(num_classes, input_shape)
model_trained = train_model(model, X_train, y_train)

Epoch 1/5
272/272 [==============================] - 3s 11ms/step - loss: 39.5522 - accuracy: 0.7511 - val_loss: 0.2271 - val_accuracy: 0.9482
Epoch 2/5
272/272 [==============================] - 3s 11ms/step - loss: 0.2038 - accuracy: 0.9540 - val_loss: 0.1359 - val_accuracy: 0.9680
Epoch 3/5
272/272 [==============================] - 3s 11ms/step - loss: 0.1279 - accuracy: 0.9701 - val_loss: 0.1165 - val_accuracy: 0.9722
Epoch 4/5
272/272 [==============================] - 3s 11ms/step - loss: 0.1065 - accuracy: 0.9756 - val_loss: 0.0741 - val_accuracy: 0.9823
Epoch 5/5
272/272 [==============================] - 3s 11ms/step - loss: 0.0977 - accuracy: 0.9774 - val_loss: 0.0640 - val_accuracy: 0.9863


In [0]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def train_and_predict(model):
  model_trained = train_model(model, X_train, y_train)
  return predict(model_trained, X_test, y_test)

In [0]:
def get_cnn_v2(num_classes, input_shape):
  return Sequential ([
     Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
     MaxPool2D(),
     Dropout(0.5),

     Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
     MaxPool2D(),
     Dropout(0.3),

     Flatten(),

     Dense(1024, activation='relu'), 
     Dropout(0.5),

     Dense(num_classes, activation='softmax')
  ])


train_and_predict( get_cnn_v2(num_classes, input_shape) )

Epoch 1/5
272/272 [==============================] - 3s 11ms/step - loss: 7.0770 - accuracy: 0.1475 - val_loss: 2.8430 - val_accuracy: 0.2459
Epoch 2/5
272/272 [==============================] - 3s 11ms/step - loss: 2.0379 - accuracy: 0.4270 - val_loss: 0.9375 - val_accuracy: 0.7521
Epoch 3/5
272/272 [==============================] - 3s 11ms/step - loss: 1.0776 - accuracy: 0.6728 - val_loss: 0.4761 - val_accuracy: 0.8871
Epoch 4/5
272/272 [==============================] - 3s 11ms/step - loss: 0.7408 - accuracy: 0.7721 - val_loss: 0.2877 - val_accuracy: 0.9334
Epoch 5/5
272/272 [==============================] - 3s 11ms/step - loss: 0.5704 - accuracy: 0.8234 - val_loss: 0.1724 - val_accuracy: 0.9602


0.8895691609977324

In [0]:
def get_cnn_v3(num_classes, input_shape):
  return Sequential ([
     Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
     Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
     MaxPool2D(),
     Dropout(0.5),

     Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
     Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
     MaxPool2D(),
     Dropout(0.3),

     Flatten(),

     Dense(1024, activation='relu'), 
     Dropout(0.5),

     Dense(num_classes, activation='softmax')
  ])


train_and_predict( get_cnn_v3(num_classes, input_shape) )

Epoch 1/5
272/272 [==============================] - 4s 16ms/step - loss: 2.4768 - accuracy: 0.4148 - val_loss: 0.6900 - val_accuracy: 0.8620
Epoch 2/5
272/272 [==============================] - 4s 15ms/step - loss: 0.7445 - accuracy: 0.7757 - val_loss: 0.1832 - val_accuracy: 0.9646
Epoch 3/5
272/272 [==============================] - 4s 15ms/step - loss: 0.4264 - accuracy: 0.8703 - val_loss: 0.0935 - val_accuracy: 0.9808
Epoch 4/5
272/272 [==============================] - 4s 15ms/step - loss: 0.2881 - accuracy: 0.9115 - val_loss: 0.0478 - val_accuracy: 0.9896
Epoch 5/5
272/272 [==============================] - 4s 15ms/step - loss: 0.2263 - accuracy: 0.9318 - val_loss: 0.0463 - val_accuracy: 0.9916


0.9739229024943311

In [0]:
def get_cnn_v4(num_classes, input_shape):
  return Sequential ([
     Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
     Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
     MaxPool2D(),
     Dropout(0.3),

     Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
     Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
     MaxPool2D(),
     Dropout(0.3),

     Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
     Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
     MaxPool2D(),
     Dropout(0.3),

     Flatten(),

     Dense(1024, activation='relu'), 
     Dropout(0.3),

     Dense(num_classes, activation='softmax')
  ])

train_and_predict( get_cnn_v4(num_classes, input_shape) )

Epoch 1/5
272/272 [==============================] - 5s 18ms/step - loss: 2.7322 - accuracy: 0.2718 - val_loss: 1.3953 - val_accuracy: 0.5444
Epoch 2/5
272/272 [==============================] - 4s 16ms/step - loss: 1.0979 - accuracy: 0.6464 - val_loss: 0.3163 - val_accuracy: 0.9042
Epoch 3/5
272/272 [==============================] - 5s 17ms/step - loss: 0.4643 - accuracy: 0.8530 - val_loss: 0.0953 - val_accuracy: 0.9728
Epoch 4/5
272/272 [==============================] - 5s 17ms/step - loss: 0.2870 - accuracy: 0.9127 - val_loss: 0.0758 - val_accuracy: 0.9792
Epoch 5/5
272/272 [==============================] - 5s 17ms/step - loss: 0.2156 - accuracy: 0.9338 - val_loss: 0.0322 - val_accuracy: 0.9910


0.9643990929705215

In [0]:
def get_cnn_v5(num_classes, input_shape):
  return Sequential ([
     Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
     Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
     MaxPool2D(),
     Dropout(0.3),

     Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
     Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
     MaxPool2D(),
     Dropout(0.3),

     Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
     Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
     MaxPool2D(),
     Dropout(0.3),

     Flatten(),

     Dense(1024, activation='relu'), 
     Dropout(0.3),

     Dense(1024, activation='relu'), 
     Dropout(0.3),

     Dense(num_classes, activation='softmax')
  ])

train_and_predict( get_cnn_v5(num_classes, input_shape) )

Epoch 1/5
272/272 [==============================] - 5s 18ms/step - loss: 2.4079 - accuracy: 0.3240 - val_loss: 1.0811 - val_accuracy: 0.6704
Epoch 2/5
272/272 [==============================] - 5s 18ms/step - loss: 0.8452 - accuracy: 0.7314 - val_loss: 0.2459 - val_accuracy: 0.9261
Epoch 3/5
272/272 [==============================] - 5s 18ms/step - loss: 0.3888 - accuracy: 0.8801 - val_loss: 0.0943 - val_accuracy: 0.9754
Epoch 4/5
272/272 [==============================] - 5s 18ms/step - loss: 0.2458 - accuracy: 0.9252 - val_loss: 0.0433 - val_accuracy: 0.9875
Epoch 5/5
272/272 [==============================] - 5s 18ms/step - loss: 0.1867 - accuracy: 0.9454 - val_loss: 0.0378 - val_accuracy: 0.9891


0.9702947845804989

RGB to GRAY

In [0]:
X_train_gray = color.rgb2gray(X_train).reshape(-1, 32, 32, 1)
X_test_gray = color.rgb2gray(X_test).reshape(-1, 32, 32, 1)


ERROR! Session/line number was not unique in database. History logging moved to new session 64


In [0]:
model =  get_cnn_v5(num_classes, (32, 32, 1))
model_trained = train_model(model, X_train_gray, y_train, params_fit={})

predict(model_trained, X_test_gray, y_test)

Epoch 1/5
272/272 [==============================] - 5s 19ms/step - loss: 2.5465 - accuracy: 0.2777 - val_loss: 0.9678 - val_accuracy: 0.7073
Epoch 2/5
272/272 [==============================] - 5s 17ms/step - loss: 0.6103 - accuracy: 0.8051 - val_loss: 0.1378 - val_accuracy: 0.9587
Epoch 3/5
272/272 [==============================] - 5s 18ms/step - loss: 0.2558 - accuracy: 0.9176 - val_loss: 0.0673 - val_accuracy: 0.9808
Epoch 4/5
272/272 [==============================] - 5s 18ms/step - loss: 0.1629 - accuracy: 0.9496 - val_loss: 0.0347 - val_accuracy: 0.9901
Epoch 5/5
272/272 [==============================] - 5s 18ms/step - loss: 0.1158 - accuracy: 0.9640 - val_loss: 0.0273 - val_accuracy: 0.9915


0.9662131519274376